<a href="https://colab.research.google.com/github/Dzikrifaza/FreshGrocie/blob/ml_dev/FreshGrocie_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Library

In [1]:
!pip install opendatasets
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.utils import Sequence

import opendatasets as od
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import splitfolders 
import pandas as pd
import os

import shutil
from PIL import Image

#Download Dataset

In [3]:
od.download("https://www.kaggle.com/datasets/muhriddinmuxiddinov/fruits-and-vegetables-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: rahmanf24
Your Kaggle Key: ··········


100%|██████████| 870M/870M [00:42<00:00, 21.7MB/s]


# Split Data

In [4]:
splitfolders.ratio('/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset(12000)/Fruits', 
                   output="/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/Fruits/", 
                   seed=42, 
                   ratio=(0.8,0.1,0.1), # training, val, test
                   group_prefix=None)

Copying files: 5997 files [00:02, 2735.93 files/s]


In [5]:
splitfolders.ratio('/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset(12000)/Vegetables', 
                   output="/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/Vegetables", 
                   seed=42, 
                   ratio=(0.8,0.1,0.1), # training, val, test
                   group_prefix=None)

Copying files: 6003 files [00:02, 2822.07 files/s]


#Merge Train and Test Data for Fruits and Vegetables 

Funtion for moving the directory

In [6]:
def move_dir(src, dest, dir_list):
  for dir_ in dir_list:
    source = os.path.join(src, dir_)

    if os.path.isdir(source):
        shutil.move(source, dest)

In [7]:
def move_subdir(source, dest, subdir):
  if not os.path.isdir(dest):
      os.mkdir(dest)

  src_path = os.path.join(source, subdir)
  for item_list in os.listdir(src_path):
      src = os.path.join(src_path, item_list)
      dst = os.path.join(dest, item_list)
      shutil.move(src, dst)

  os.rmdir(src_path)

Train Data:

Taking the data from fruits and vegetables and combine them into 1 training data

In [8]:
source_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/Fruits'

dir_list = ['train']

dest_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset'
dest = os.path.join(dest_path, 'train')

move_dir(source_path, dest, dir_list)

In [9]:
source_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/Vegetables'

subdir = 'train'

dest_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/train'

move_subdir(source_path, dest_path, subdir)

Validation Data:

Taking the data from fruits and vegetables and combine them into 1 validation data

In [10]:
source_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/Fruits'

dir_list = ['val']

dest_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset'
dest = os.path.join(dest_path, 'val')

move_dir(source_path, dest, dir_list)

In [11]:
source_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/Vegetables'

subdir = 'val'

dest_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/val'

move_subdir(source_path, dest_path, subdir)

Test Data:

Taking the data from fruits and vegetables and combine them into 1 testing data

In [12]:
source_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/Fruits'
 
dir_list = ['test']
 
dest_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset'
dest = os.path.join(dest_path, 'test')

move_dir(source_path, dest, dir_list)

In [13]:
source_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/Vegetables'

subdir = 'test'

dest_path = '/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/test'

move_subdir(source_path, dest_path, subdir)

#Assign

In [14]:
train_FileNames = os.path.join('/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/train')
val_FileNames = os.path.join('/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/val')
test_FileNames = os.path.join('/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset/test')

Convert the RGBA

In [15]:
directory = "/content/fruits-and-vegetables-dataset/Fruits_Vegetables_Dataset"

for root, _, files in os.walk(directory):
    for file in files:
        if file.endswith(".png") or file.endswith(".webp"):
            image_path = os.path.join(root, file)

            with Image.open(image_path) as img:
                if img.mode == "P" and "transparency" in img.info:
                    img = img.convert("RGBA")

                save_path = os.path.splitext(image_path)[0] + ".png"
                if file.endswith(".webp"):
                    save_path = os.path.splitext(image_path)[0] + ".jpg"

                img.save(save_path)

#Model

In [16]:
import tensorflow as tf

def create_model():
  model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(20, activation='softmax')
    ])

  return model

In [17]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

#Normalization

In [18]:
tf.random.set_seed(42)

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   brightness_range=(0.2, 0.8),
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_FileNames,  
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_FileNames,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

Found 9582 images belonging to 20 classes.
Found 1192 images belonging to 20 classes.


#Compile and Train the model

In [20]:
model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.SGD(learning_rate=9e-6, momentum=0.9),
                metrics=['accuracy'])

In [21]:
# Train the model
inception_v3 = model.fit(
    train_generator,
    batch_size = 20,
    steps_per_epoch = 480, 
    epochs= 10,
    validation_data=validation_generator,
    validation_steps=59.6,
    verbose=2)

Epoch 1/10
480/480 - 121s - loss: 2.9969 - accuracy: 0.0611 - val_loss: 2.9994 - val_accuracy: 0.0512 - 121s/epoch - 252ms/step
Epoch 2/10
480/480 - 112s - loss: 2.9968 - accuracy: 0.0563 - val_loss: 2.9989 - val_accuracy: 0.0512 - 112s/epoch - 234ms/step
Epoch 3/10
480/480 - 108s - loss: 2.9966 - accuracy: 0.0570 - val_loss: 2.9984 - val_accuracy: 0.0495 - 108s/epoch - 226ms/step
Epoch 4/10
480/480 - 107s - loss: 2.9963 - accuracy: 0.0537 - val_loss: 2.9980 - val_accuracy: 0.0461 - 107s/epoch - 223ms/step
Epoch 5/10
480/480 - 107s - loss: 2.9962 - accuracy: 0.0555 - val_loss: 2.9977 - val_accuracy: 0.0445 - 107s/epoch - 223ms/step
Epoch 6/10
480/480 - 108s - loss: 2.9965 - accuracy: 0.0540 - val_loss: 2.9974 - val_accuracy: 0.0436 - 108s/epoch - 224ms/step
Epoch 7/10
480/480 - 112s - loss: 2.9961 - accuracy: 0.0532 - val_loss: 2.9972 - val_accuracy: 0.0445 - 112s/epoch - 233ms/step
Epoch 8/10
480/480 - 107s - loss: 2.9962 - accuracy: 0.0564 - val_loss: 2.9969 - val_accuracy: 0.0445 - 